# 可微量子架构搜索

## 概述

本教程演示了如何利用 TensorCircuit 提供的高级计算功能，例如 ``jit`` 和 ``vmap`` 来超级有效地模拟可微量子架构搜索（DQAS）算法，其中具有不同结构的量子电路的集合可以同时编译模拟。
[WIP note]

## 设置

In [1]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf

In [2]:
K = tc.set_backend("tensorflow")
ctype, rtype = tc.set_dtype("complex128")

## 问题描述

任务是找到 GHZ 状态的状态准备电路 $\vert \text{GHZ}_N\rangle = \frac{1}{\sqrt{2}}\left(\vert 0^N\rangle +\vert 1^N\rangle \right)$。我们为 $N=2$ 演示准备了一个包含 rx0、rx1、ry0、ry1、rz0、rz1、cnot01、cnot10 的门池。 在八个门中，有六个是参数化的。

In [3]:
def rx0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix, K.eye(2)
    )


def rx1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix
    )


def ry0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix, K.eye(2)
    )


def ry1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix
    )


def rz0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix, K.eye(2)
    )


def rz1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix
    )


def cnot01():
    return K.cast(K.convert_to_tensor(tc.gates._cnot_matrix), ctype)


def cnot10():
    return K.cast(
        K.convert_to_tensor(
            np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        ),
        ctype,
    )


ops_repr = ["rx0", "rx1", "ry0", "ry1", "rz0", "rz1", "cnot01", "cnot10"]

In [4]:
n, p, ch = 2, 3, 8
# 量子比特数、层数、操作池大小

target = tc.array_to_tensor(np.array([1, 0, 0, 1.0]) / np.sqrt(2.0))
# 目标波函数，我们这里使用 GHZ2 状态作为目标函数


def ansatz(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    loss = K.sum(K.abs(target - s))
    return loss


vag1 = K.jit(K.vvag(ansatz, argnums=0, vectorized_argnums=1))

## 概率系综方法

这种方法更加实用和实验相关，并且与参考文献 1 中描述的算法相同，尽管我们在这里使用高级 vmap 来加速具有不同结构的电路的仿真。

In [5]:
structures = K.zeros([p, ch], rtype)


def sampling_from_structure(structures, batch=1):
    prob = K.softmax(K.real(structures), axis=-1)
    return np.array([np.random.choice(ch, p=K.numpy(prob[i])) for i in range(p)])


@K.jit
def best_from_structure(structures):
    return K.argmax(structures, axis=-1)


@K.jit
def nmf_gradient(structures, oh):
    """
    根据朴素平均场概率模型计算蒙特卡洛梯度
    """
    choice = K.argmax(oh, axis=-1)
    prob = K.softmax(K.real(structures), axis=-1)
    indices = K.transpose(K.stack([K.cast(tf.range(p), "int64"), choice]))
    prob = tf.gather_nd(prob, indices)
    prob = K.reshape(prob, [-1, 1])
    prob = K.tile(prob, [1, ch])

    return tf.tensor_scatter_nd_add(
        tf.cast(-prob, dtype=ctype),
        indices,
        tf.ones([p], dtype=ctype),
    )


nmf_gradient_vmap = K.vmap(nmf_gradient, vectorized_argnums=1)

In [6]:
verbose = False
epochs = 400
batch = 256
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.12))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
avcost1 = 0
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    batched_stuctures = K.onehot(
        np.stack([sampling_from_structure(stp) for _ in range(batch)]), num=8
    )
    infd, gnnp = vag1(nnp, batched_stuctures)
    gs = nmf_gradient_vmap(structures, batched_stuctures)  # \nabla lnp
    gstp = [K.cast((infd[i] - avcost2), ctype) * gs[i] for i in range(infd.shape[0])]
    gstp = K.real(K.sum(gstp, axis=0) / infd.shape[0])
    avcost1 = K.sum(infd) / infd.shape[0]
    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)

    if epoch % 40 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(avcost1),
        )

        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.4212375301510944
best candidates so far: ['cnot01', 'ry1', 'rz1']
corresponding weights for each gate: [0.0, 0.05877275111731101, -0.04664915789120658]
----------epoch 40-----------
batched average loss:  1.0250998670008267
best candidates so far: ['cnot01', 'rz0', 'cnot01']
corresponding weights for each gate: [0.0, 0.024759131232497896, 0.0]
----------epoch 80-----------
batched average loss:  0.8947977316164302
best candidates so far: ['cnot01', 'rz0', 'cnot01']
corresponding weights for each gate: [0.0, 0.025322026393257882, 0.0]
----------epoch 120-----------
batched average loss:  0.03468271497984207
best candidates so far: ['cnot01', 'ry0', 'cnot01']
corresponding weights for each gate: [0.0, -0.7656295586663117, 0.0]
----------epoch 160-----------
batched average loss:  0.012709563347046916
best candidates so far: ['cnot01', 'ry0', 'cnot01']
corresponding weights for each gate: [0.0, -0.7675056347152348, 0.0]
----------epoch

## 直接优化结构参数

无论如何，由于我们是用数值模拟，所以可以直接优化结构参数，省略超级电路是否是幺正的，这种方法在某些场景下可以更快更可靠。

In [7]:
def ansatz2(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.softmax(structures, axis=-1)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    s /= K.norm(s)
    loss = K.sum(K.abs(target - s))
    return loss


vag2 = K.jit(K.value_and_grad(ansatz2, argnums=(0, 1)))

In [8]:
verbose = True
epochs = 700
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.05, 200, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.04))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
for epoch in range(epochs):

    infd, (gnnp, gstp) = vag2(nnp, stp)

    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)
    if epoch % 70 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(infd),
        )
        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.3024341605187928
strcuture parameter: 
 [[ 0.00265054  0.04495954  0.05265605  0.04751008  0.03309468  0.02743368
   0.03382795 -0.06647121]
 [ 0.03544281  0.03207712  0.03629811  0.0266235   0.03264895  0.03198189
   0.03505167 -0.03449981]
 [ 0.0304648   0.07042194  0.03075206  0.02515865  0.02984363  0.00955019
   0.07527341 -0.05831911]] 
 network parameter: 
 [[-0.0380125   0.0688923   0.04393423  0.04205065  0.06243917  0.03672062]
 [-0.05277717  0.04834309  0.05176114  0.07030034  0.02983666  0.04821408]
 [-0.04095011  0.0393773   0.03383929  0.06559557  0.03458135  0.02436751]]
best candidates so far: ['ry0', 'ry0', 'cnot01']
corresponding weights for each gate: [0.043934227235354874, 0.05176113831452516, 0.0]
----------epoch 70-----------
batched average loss:  1.0078726220234666
strcuture parameter: 
 [[ 0.34119556  0.37154559  0.2781548   0.37689646  1.79624262  1.78939153
   1.79791154 -0.3958881 ]
 [-1.04375011 -0.12801

## 最后的微调

对于获得的电路布局，我们可以进一步调整电路权重，使目标函数更接近于零。

In [9]:
chosen_structure = K.onehot(np.array([2, 4, 6]), num=8)
chosen_structure = K.reshape(chosen_structure, [1, p, ch])
chosen_structure

<tf.Tensor: shape=(1, 3, 8), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.]]], dtype=float32)>

In [10]:
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-3))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
verbose = True
epochs = 600
for epoch in range(epochs):
    infd, gnnp = vag1(nnp, chosen_structure)
    nnp = network_opt.update(gnnp, nnp)
    if epoch % 60 == 0 or epoch == epochs - 1:
        print(epoch, "loss: ", K.numpy(infd[0]))

0 loss:  1.004872758871745
60 loss:  0.9679200431227549
120 loss:  0.9091748060127385
180 loss:  0.8302632245154631
240 loss:  0.73297561645977
300 loss:  0.6183436622858383
360 loss:  0.4874390992051161
420 loss:  0.34168453047914643
480 loss:  0.18299822849737177
540 loss:  0.013923344772669728
599 loss:  0.0016133833518836463


## 参考资料

1. https://arxiv.org/pdf/2010.08561.pdf